In [20]:
# the generator used the part of features to make all data
# We suppose that it is a GAN, DCGAN

from Dataloaders import dataloader_cifar10
import datetime
from Models import mobilenetv2, mobilenetv3, resnet, generator, encoder_client
import numpy as np
import os
import scipy
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from Utils import utils

in_ch = 32
g_rate = [0.4]
# logger
stime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
logger = utils.APLogger('./Logs/train_generator_' + stime + '.log')
logger.write('Dataset: CIFAR-10\n, Model: mobilenetv2\n, Generator: GeneratorV1\n, Encoder: 2layerCNN\n, Batch: 128\n, Epochs: 100\n, Seed: 2024\n')
logger.write('Rate: %.1f\n'%g_rate[0])

# client, and server model
client_model, server_model = mobilenetv2.mobilenetv2_splitter(num_classes=10, weight_root='./Weights/cifar-10/', partition=-1)
client_model.eval()
server_model.eval()
client_model = client_model.to('cuda:0')
server_model = server_model.to('cuda:0')
# for param in client_model.parameters():
#     param.requires_grad = False
# for param in server_model.parameters():
#     param.requires_grad = False

# dataloader using the 
train, _, val = dataloader_cifar10.Dataloader_cifar10_val(train_batch=128, test_batch=100, seed=2024)

# we have 3 generators for 3 discriminators
Generators = []

for i in range(len(g_rate)):
    Generators.append(generator.Generator(int(in_ch*g_rate[i]), hiddensize=32, outputsize=32).cuda())
    Generators[i] = Generators[i].cuda()

Encoders = []
for i in range(len(g_rate)):
    Encoders.append(encoder_client.Encoder_Client(in_ch=in_ch, out_ch=int(g_rate[i]*in_ch)).cuda())
    Encoders[i] = Encoders[i].cuda()

criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()


for i in range (len(Generators)):

    netG = Generators[i]
    netG.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/generator/generator_0.4_2024_07_08_03_17_07.pth'))
    netG = netG.cuda()

    netE = Encoders[i]
    netE.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/encoder/encoder_0.4_2024_07_08_03_17_07.pth'))
    netE = netE.cuda()

    netG.eval()
    netE.eval()
    
    with torch.no_grad():

        for j, data in enumerate(val):
            img, label = data
            img, label = img.cuda(), label.cuda()
            
            out = client_model(img).detach()
            
            out_max = torch.max(out)
            out_min = torch.min(out)

            out = (out - out_min) / (out_max - out_min)

            # train the encoder
            out = netE(out)
            # train the generator
            out = netG(out)

            out = out * (out_max - out_min) + out_min
            # train the encoder            
            out = server_model(out)
            out = torch.argmax(out, dim=1)
            # change it to float
            print(out)
            print(label)
            break

        


Files already downloaded and verified
Files already downloaded and verified
tensor([7, 4, 0, 9, 4, 1, 3, 3, 7, 7, 1, 1, 7, 3, 4, 9, 0, 9, 4, 0, 9, 3, 9, 8,
        7, 9, 1, 0, 9, 9, 4, 7, 9, 0, 9, 1, 3, 1, 3, 8, 8, 0, 7, 1, 6, 3, 7, 2,
        4, 6, 9, 8, 3, 1, 7, 8, 7, 1, 3, 6, 0, 8, 0, 7, 1, 9, 7, 1, 3, 2, 1, 3,
        4, 1, 7, 9, 0, 7, 1, 7, 7, 3, 3, 9, 1, 9, 1, 2, 4, 1, 8, 7, 8, 7, 8, 2,
        3, 3, 5, 7, 8, 3, 8, 1, 3, 8, 2, 0, 0, 7, 0, 1, 8, 8, 5, 7, 9, 3, 4, 7,
        1, 9, 3, 3, 2, 7, 7, 7], device='cuda:0')
tensor([2, 6, 3, 1, 2, 0, 6, 2, 4, 5, 1, 1, 5, 8, 5, 9, 0, 9, 2, 0, 8, 3, 2, 0,
        7, 1, 1, 8, 8, 1, 3, 5, 9, 8, 8, 9, 5, 1, 3, 8, 8, 0, 5, 1, 6, 6, 4, 2,
        7, 6, 1, 8, 7, 1, 3, 3, 9, 1, 3, 6, 0, 8, 0, 9, 9, 0, 7, 1, 4, 6, 1, 2,
        2, 1, 2, 9, 0, 1, 1, 5, 3, 3, 6, 9, 1, 7, 1, 3, 3, 9, 0, 7, 0, 4, 0, 4,
        3, 2, 5, 7, 9, 7, 1, 3, 5, 8, 6, 0, 0, 5, 0, 9, 8, 8, 5, 3, 1, 6, 2, 5,
        2, 9, 3, 5, 4, 6, 7, 5], device='cuda:0')


In [45]:
# the generator used the part of features to make all data
# We suppose that it is a GAN, DCGAN

from Dataloaders import dataloader_cifar10, dataloader_imagenet
import datetime
from Models import mobilenetv2, mobilenetv3, resnet, generator, encoder_client
import numpy as np
import os
import scipy
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from Utils import utils

in_ch = 32
g_rate = [0.6]
# logger
stime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
logger = utils.APLogger('./Logs/train_generator_' + stime + '.log')
logger.write('Dataset: CIFAR-10\n, Model: mobilenetv2\n, Generator: GeneratorV1\n, Encoder: 2layerCNN\n, Batch: 128\n, Epochs: 100\n, Seed: 2024\n')
logger.write('Rate: %.1f\n'%g_rate[0])

# client, and server model
client_model, server_model = mobilenetv2.mobilenetv2_splitter(num_classes=10, weight_root='./Weights/cifar-10/', partition=-1)
# client_model, server_model = mobilenetv2.mobilenetv2_splitter(num_classes=1000, weight_root='./Weights/imagenet/', partition=-1)
client_model.eval()
server_model.eval()
client_model = client_model.to('cuda:0')
server_model = server_model.to('cuda:0')
# for param in client_model.parameters():
#     param.requires_grad = False
# for param in server_model.parameters():
#     param.requires_grad = False

# dataloader using the 
train, _, val = dataloader_cifar10.Dataloader_cifar10_val(train_batch=128, test_batch=100, seed=2024)
# train, _, val, _ = dataloader_imagenet.Dataloader_imagenet_integrated()

# we have 3 generators for 3 discriminators
Generators = []

for i in range(len(g_rate)):
    Generators.append(generator.Generator(int(in_ch*g_rate[i]), hiddensize=32, outputsize=32).cuda())
    Generators[i] = Generators[i].cuda()

Encoders = []
for i in range(len(g_rate)):
    Encoders.append(encoder_client.Encoder_Client(in_ch=in_ch, out_ch=int(g_rate[i]*in_ch)).cuda())
    Encoders[i] = Encoders[i].cuda()

criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()


for i in range (len(Generators)):

    netG = Generators[i]
    # netG.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/generator/generator_0.6_2024_07_08_03_51_38.pth'))
    netG = netG.cuda()

    netE = Encoders[i]
    # netE.load_state_dict(torch.load('/home/tonypeng/Workspace1/adaptfilter/Adaptfilter/Weights/cifar-10/encoder/encoder_0.6_2024_07_08_03_51_38.pth'))
    netE = netE.cuda()

    netG.eval()
    netE.eval()
    
    with torch.no_grad():
        total_time = 0
        total_time2 = 0

        for j, data in enumerate(val):
            imgs, labels = data
            for k in range (imgs.size(0)):
                img, label = imgs[k], labels[k]
                img = img.unsqueeze(0)
                label = label.unsqueeze(0)
                img, label = img.cuda(), label.cuda()
                
                out = client_model(img).detach()
                
                out_max = torch.max(out)
                out_min = torch.min(out)

                out = (out - out_min) / (out_max - out_min)

                # train the encoder
                out = netE(out)
                # train the generator
                time1 = time.time()
                out = netG(out)

                out = out * (out_max - out_min) + out_min
                # train the encoder
                # time1 = time.time()            
                out = server_model(out)
                out = torch.argmax(out, dim=1)
                time2 = time.time()
                total_time += time2 - time1
                # change it to float

                time1 = time.time()
                out = client_model(img).detach()
                out = server_model(out)
                out = torch.argmax(out, dim=1)
                time2 = time.time()
                total_time2 += time2 - time1
            break
        print(k)
print('Time: %.4f'%(total_time / (k+1)))
print('Time2: %.4f'%(total_time2 / (k+1)))


Files already downloaded and verified
Files already downloaded and verified
127
Time: 0.0030
Time2: 0.0028
